In [ ]:
import requests
import csv

def query_wikidata():
    # Define the SPARQL query to fetch meteorites discovered in United States, Mexico, or Canada
    sparql_query = """
    SELECT ?meteorite ?meteoriteLabel ?discovery_time
    WHERE
    {
        ?meteorite wdt:P31 wd:Q60186;  # instance of meteorite
                   wdt:P17 ?country;    # located in country
                   wdt:P575 ?discovery_time.  # time of discovery
        FILTER (?country IN (wd:Q30, wd:Q96, wd:Q16))  # United States, Mexico, Canada
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    # Wikidata API endpoint
    api_url = "https://query.wikidata.org/sparql"

    # HTTP GET request to Wikidata API
    response = requests.get(api_url, params={'format': 'json', 'query': sparql_query})

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()['results']['bindings']
    else:
        print("Error fetching data from Wikidata:", response.text)
        return None

def save_to_csv(data):
    # Define CSV file name
    csv_file = 'meteorites_data.csv'

    # Write fetched data to CSV file
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write header row
        writer.writerow(["Meteorite ID", "Meteorite Name", "Discovery Time"])
        # Write data rows
        for item in data:
            meteorite_id = item['meteorite']['value'].split('/')[-1]
            meteorite_name = item['meteoriteLabel']['value']
            discovery_time = item['discovery_time']['value']
            writer.writerow([meteorite_id, meteorite_name, discovery_time])

def main():
    # Query Wikidata
    result = query_wikidata()
    if result:
        # Save data to CSV
        save_to_csv(result)
        print("Data saved to meteorites_data.csv")
    else:
        print("Failed to fetch data from Wikidata")

if __name__ == "__main__":
    main()
